In [1]:
import csv2pdf
import os
import smtplib
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from dateutil.relativedelta import relativedelta
from email.mime.base import MIMEBase
from email import encoders
from numpy import average
import pandas as pd
import numpy as np
import math
import psycopg2
import smtplib
import pandas.io.sql as psql
from ftplib import FTP_TLS
import ssl
import pandas as pd
import datetime
from psycopg2.extensions import STATUS_BEGIN, STATUS_READY
from datetime import datetime
import dateutil.relativedelta
import warnings
import argparse
import csv
warnings.filterwarnings("ignore")

In [2]:
def sendEmail(names, files, subject):
    # sending emails
    fromaddr = "reports@scalptrade.com"
    toaddr = names

    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = ", ".join(toaddr)
    msg['Subject'] = '{}'.format(subject)

    # Attach files
    for file_path in files:
        with open(file_path, 'rb') as f:
            # Convert CSV to PDF and attach PDF to email if file is a CSV
            if file_path.endswith('.csv'):
                csv_data = f.read()
                pdf_data = csv2pdf.csv2pdf(csv_data)
                pdf_filename = os.path.splitext(file_path)[0] + '.pdf'
                with open(pdf_filename, 'wb') as pdf_file:
                    pdf_file.write(pdf_data)

                attachment = MIMEApplication(pdf_data, _subtype='pdf')
                attachment.add_header('content-disposition', 'attachment', filename=pdf_filename)
                msg.attach(attachment)

            # Attach other files as is
            else:
                attachment = MIMEApplication(f.read(), _subtype='octet-stream')
                attachment.add_header('content-disposition', 'attachment', filename=os.path.basename(file_path))
                msg.attach(attachment)

    # Sending email
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login(fromaddr, "sc@lptrade")
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)

    # Delete PDF files generated from CSVs
    for file_path in files:
        if file_path.endswith('.csv'):
            pdf_filename = os.path.splitext(file_path)[0] + '.pdf'
            os.remove(pdf_filename)